In [2]:
import pandas as pd

##### Reading data

In [3]:
#filename = 'D:\\Backup\\ISB CBA\\Capstone\\summary data\\transaction_summary.csv'

#data=pd.read_csv(filename)
#d={'quantity':'count','sales_quantity':'sum','purchase_quantity':'sum','transfer_quantity':'sum'}
#chunksize = 10 ** 5
#rows=0
#df_north = pd.DataFrame()
#for chunk in pd.read_csv(filename, chunksize=chunksize):
# 
#    n=chunk[chunk['state']=='ST03']
#    df_north=pd.concat([df_north,n])
#    rows+=chunk.shape[0]
    #print (rows)

In [3]:
data_path = '/Users/parulgaba/Desktop/Capstone-Ethos/ethos-retail-model/data/'
filename = data_path + 'summary_sorted_area_02.csv'
df_north=pd.read_csv(filename)

/Users/parulgaba/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_north.shape

(1103480, 51)

In [5]:
df_north.columns

Index(['state', 'location_code', 'item_no', 'week', 'closing_date', 'region',
       'brand', 'quantity', 'purchase_quantity', 'transfer_quantity',
       'sales_quantity', 'purchase_cost_amount', 'purchase_mrp',
       'purchase_date', 'stock_prevailing_mrp', 'store_in',
       'product_group_code', 'transfer_cost_amount', 'sales_department',
       'days_to_sell', 'num_of_customers', 'total_price', 'line_discount',
       'crm_line_discount', 'discount', 'tax', 'cost', 'billing',
       'contribution', 'trade_incentive', 'trade_incentive_value',
       'total_contribution', 'store_type', 'store_location', 'city_type',
       'available_quantity', 'week_no', 'year', 'case_size', 'case_size_range',
       'gender', 'movement', 'material', 'dial_color', 'strap_type',
       'strap_color', 'precious_stone', 'glass', 'case_shape', 'watch_type',
       'area_code'],
      dtype='object')

In [6]:
#extracting specific columns

col=['location_code', 'item_no', 'state', 'region',
       'brand', 'stock_prevailing_mrp', 'store_type', 'store_location', 'city_type',
       'available_quantity', 'week_no', 'year', 'case_size_range',
       'gender', 'movement', 'material', 'dial_color', 'strap_type',
       'strap_color', 'precious_stone', 'glass', 'case_shape', 'watch_type','billing','sales_quantity']

df_north_final=df_north.loc[:,col]
df_north_final.fillna(0, inplace=True)

df_north_final['item_no']=df_north_final['item_no'].astype(str)
#df_north_final['case_size']=df_north_final['case_size'].astype(str)
df_north_final['case_shape']=df_north_final['case_shape'].astype(str)

#### Defining a function for doing pareto analysis on features

The function aggregates all levels of a categorical features that account for ~10% by Sales billings into a new level called "others". Features with less than 10 levels are not considered for pareto analysis.

In [7]:
def pareto(cols):
    lst=[]
    for col in cols:
                
        series=df_north_final.fillna(0).groupby([col]).agg({'billing':'sum'}).sort_values('billing',ascending=False)
        mask=series.cumsum()/series.sum()>0.9 
        #nos=mask.value_counts()[1]
        mask=mask.iloc[:,0]
        levels=len(df_north_final[col].unique())
        if levels>10:
            df_north_final[col] = np.where(df_north_final[col].isin(series[mask].index),'Other',df_north_final[col])         
        new_levels=len(df_north_final[col].unique())

        freq=df_north_final[col].value_counts()/df_north_final[col].value_counts().sum()*100
        freq=freq.round(2)

        sale_qty=df_north_final.groupby([col]).agg({'sales_quantity':'sum'}).sort_values('sales_quantity',ascending=False)
        sale_qty=sale_qty/sale_qty.sum()*100
        sale_qty=sale_qty.round(2)
        try:
            Other_Sales_Qty=sale_qty['sales_quantity']['Other']
        except:
            Other_Sales_Qty=0
        
        mrp=df_north_final.groupby([col]).agg({'stock_prevailing_mrp':'sum'}).sort_values('stock_prevailing_mrp',ascending=False)
        mrp=mrp/mrp.sum()*100
        mrp=mrp.round(2)
        try:
            Other_MRP=mrp['stock_prevailing_mrp']['Other']
        except:
            Other_MRP=0
        
        #comparison=mrp.merge(sale_qty, left_index=True, right_index=True)
        lst.append([col.upper(),levels, new_levels,Other_MRP,Other_Sales_Qty])
        #print ("%s-Originally %d levels,combined %d levels into 'Other'.New Levels %d.By MRP,Other is %2.1f and by sale qty others is %2.1f"%(col.upper(),levels, levels-new_levels, new_levels,mrp['stock_prevailing_mrp']['Other'],sale_qty['sales_quantity']['Other']))
    
    cols=['Feature', 'Orig Levels', 'New Levels', 'Other%(MRP)', 'Other%(Sales Qty)']
    df = pd.DataFrame(lst, columns=cols)
    df=df.set_index("Feature")
    
    return df

In [8]:
import numpy as np
#cols=['brand','case_size', 'case_size_range', 'gender', 'material', 'dial_color', 'strap_type', 'strap_color','precious_stone', 'glass', 'watch_type']
cols=['case_size_range', 'gender','movement', 'material', 'dial_color', 'strap_type', 'strap_color','precious_stone', 'glass', 'case_shape', 'watch_type']
pareto(cols)

,Orig Levels,New Levels,Other%(MRP),Other%(Sales Qty)
Feature,,,,
CASE_SIZE_RANGE,13,6,20.98,22.70
GENDER,3,3,0.00,0.00
MOVEMENT,10,10,0.00,0.00
MATERIAL,80,10,14.10,15.51
DIAL_COLOR,63,11,13.27,10.56
STRAP_TYPE,81,11,12.30,14.97
STRAP_COLOR,56,7,15.45,21.41
PRECIOUS_STONE,9,9,0.00,0.00
GLASS,8,8,0.00,0.00


### Modelling

In [9]:
#sorting by week_no and year
df_north_final=df_north_final.sort_values(['week_no', 'year'], ascending=[True, False])

print('Done')

Done


In [10]:
#creatign dummy variables
cols=['brand','state','region', 'store_type', 'store_location', 'city_type',
       'case_size_range', 'gender', 'movement', 'material', 'dial_color',
       'strap_type', 'strap_color', 'precious_stone', 'glass', 'case_shape',
       'watch_type', 'week_no', 'year']
df_north_dummies=pd.get_dummies(data=df_north_final, columns=cols)

print('Done')

Done


In [11]:
df_north_dummies.shape

(1103480, 274)

In [12]:
import yaml

In [28]:
#!pip install xgboost
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [13]:
df_north_dummies.head()

,location_code,item_no,stock_prevailing_mrp,available_quantity,billing,sales_quantity,brand_0,brand_B003,brand_B007,brand_B008,...,week_no_W47,week_no_W48,week_no_W49,week_no_W50,week_no_W51,week_no_W52,week_no_W53,year_FY1718,year_FY1819,year_FY1920
13053,S03,5127095,24990.0,1,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
13054,S03,5154203,196000.0,1,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
13055,S03,5185559,111000.0,1,0.0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
13056,S03,5188116,252000.0,3,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
13057,S03,5188553,8295.0,1,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [14]:
y=df_north_dummies.loc[:, df_north_dummies.columns == 'sales_quantity']
X=df_north_dummies.drop(columns =['sales_quantity','location_code','item_no'])

In [15]:
#checking for duplicate column names
duplicate_columns = df_north_dummies.columns[df_north_dummies.columns.duplicated()]
duplicate_columns

Index([], dtype='object')

In [16]:
#performing train and test split on data

split=0.2
test=int(len(X)*split)
train=len(X)-test
X_train=X.head(train)
y_train=y.head(train)
X_test=X.tail(test)
y_test=y.tail(test)

In [18]:
y_test

,sales_quantity
880569,0
880570,0
880571,0
880572,0
880573,0
880574,0
880575,0
880576,0
880577,0
880578,0


#### Modelling using xgboost

In [34]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,max_depth = 5, alpha = 10, n_estimators = 10)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.581572


In [36]:
from sklearn.metrics import r2_score
r2_score(y_test,preds)

-0.060883552713791866

##### Tuning hyperparameters

In [37]:
params = {'colsample_bytree': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
          'subsample': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
          'colsample_bylevel':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
          'min_child_weight ':[1,3,5,7] ,
          'reg_lambda':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
          'n_estimator':[3,5,7,10],
          'learning_rate': [0.05,0.1,0.15,0.2,0.25,0.3],
          'max_depth': [3,4,5,6,7,8,10,12,15], 
          'reg_alpha': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
         }

In [38]:
params = {'colsample_bytree': [0.1,0.2],
          #'subsample': [0.1,0.2,0.3,0.4],
          #'colsample_bylevel':[0.1,0.2,0.3,0.4],
          #'min_child_weight ':[1,3,5,7] ,
          'reg_lambda':[0.1,0.2,0.3]
          #'n_estimator':[3,5,],
          #'learning_rate': [0.05,0.1,0.15,0.2,0.25,0.3],
          #'max_depth': [3,4,5,6,7,8,10,12,15], 
          #'reg_alpha': [0.1,0.2,0.3]
         }

In [39]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror')
scoring='r2'
#scoring='neg_mean_squared_error'
random_search=RandomizedSearchCV(xg_reg,param_distributions=params,scoring=scoring,n_iter=20,cv=5,verbose=3)

In [40]:
random_search.fit(X_train,y_train)

/Users/parulgaba/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:282: UserWarning: The total space of parameters 6 is smaller than n_iter=20. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] reg_lambda=0.1, colsample_bytree=0.1 ............................
[CV]  reg_lambda=0.1, colsample_bytree=0.1, score=0.421, total= 2.0min
[CV] reg_lambda=0.1, colsample_bytree=0.1 ............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.0min remaining:    0.0s


[CV]  reg_lambda=0.1, colsample_bytree=0.1, score=0.271, total= 2.0min
[CV] reg_lambda=0.1, colsample_bytree=0.1 ............................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.9min remaining:    0.0s


[CV]  reg_lambda=0.1, colsample_bytree=0.1, score=0.470, total= 1.9min
[CV] reg_lambda=0.1, colsample_bytree=0.1 ............................
[CV]  reg_lambda=0.1, colsample_bytree=0.1, score=0.117, total= 2.0min
[CV] reg_lambda=0.1, colsample_bytree=0.1 ............................
[CV]  reg_lambda=0.1, colsample_bytree=0.1, score=-0.198, total= 2.1min
[CV] reg_lambda=0.2, colsample_bytree=0.1 ............................
[CV]  reg_lambda=0.2, colsample_bytree=0.1, score=0.416, total= 1.9min
[CV] reg_lambda=0.2, colsample_bytree=0.1 ............................
[CV]  reg_lambda=0.2, colsample_bytree=0.1, score=0.268, total= 1.9min
[CV] reg_lambda=0.2, colsample_bytree=0.1 ............................
[CV]  reg_lambda=0.2, colsample_bytree=0.1, score=0.468, total= 3.9min
[CV] reg_lambda=0.2, colsample_bytree=0.1 ............................
[CV]  reg_lambda=0.2, colsample_bytree=0.1, score=0.104, total= 2.0min
[CV] reg_lambda=0.2, colsample_bytree=0.1 ............................
[CV] 

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 79.6min finished


RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, gamma=None,
                                          gpu_id=None, importance_type='gain',
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          n_estimators=100, n_jobs=None,
                                          num_parallel_tree=None,
                                          random_state=None, reg_alpha=None,
                                       

In [ ]:
random_search.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.2, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=0.1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [ ]:
random_search.best_params_

{'reg_lambda': 0.1, 'colsample_bytree': 0.2}

In [17]:
print('Done 2')

Done 2


In [ ]:
best_gb=xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.4,
             colsample_bynode=1, colsample_bytree=0.9, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.2, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=np.nan,
             monotone_constraints='()', n_estimator=5, n_estimators=100,
             n_jobs=4, num_parallel_tree=1, objective='reg:squarederror',
             random_state=0, reg_alpha=0.4, reg_lambda=0.1, scale_pos_weight=1,
             subsample=0.3, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [ ]:
from sklearn.model_selection import cross_val_score as cvs
score=cvs(best_gb,X_train,y_train,cv=5,scoring='r2')
print('Done')
print(score)

[23:49:58] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { n_estimator } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[23:52:14] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { n_estimator } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[23:54:33] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { n_estimator } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but

In [48]:
score

array([0.23704568, 0.08570453, 0.45662306, 0.18019469, 0.32374679])

In [49]:
score.mean()

0.2566629502603503

In [50]:
best_gb.fit(X_train,y_train)
preds = best_gb.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

[00:02:39] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { n_estimator } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


RMSE: 0.537640


In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test,preds)

In [ ]:
import numpy as np
import statsmodels.api as sm

from statsmodels.genmod.generalized_estimating_equations import GEE
from statsmodels.genmod.cov_struct import (Exchangeable,
    Independence,Autoregressive)
from statsmodels.genmod.families import Poisson

In [54]:
"""!pip install statsmodels

X_train=X.head(train)
y_train=y.head(train)
X_test=X.tail(test)
y_test=y.tail(test)
"""

In [68]:
expr = """sales_quantity ~ ."""


In [ ]:
#Using the statsmodels GLM class, train the Poisson regression model on the training data set.
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

In [ ]:
zip_training_results = sm.ZeroInflatedPoisson(endog=y_train, exog=X_train, exog_infl=X_train, inflation='logit').fit()

In [ ]:
print(zip_training_results.summary())

In [ ]:
print('Done')